In [ ]:
# default_exp data

In [ ]:
#all_slow

# Data
> Classes and functions for managing data

In [ ]:
#hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
#hide
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

In [ ]:
#export
from lemonpie.basics import *
from lemonpie.preprocessing.transform import *
from fastai.imports import *
import copy

In [ ]:
#hide
from nbdev.showdoc import *

## Split

- Split is already done in the raw data before vocab creation.
- The following class just to hold everything together

In [ ]:
#export
class EHRDataSplits():
    '''Class to hold the PatientList splits; defaults to loading 0 to 20 years age span'''
    def __init__(self, path, age_start=0, age_stop=20, age_in_months=False):
        self.train, self.valid, self.test = self._load_splits(path, age_start, age_stop, age_in_months)
    
    def _load_splits(self, path, age_start, age_stop, age_in_months):
        '''Load splits of preprocessed `PatientList`s from persistent store using path'''
        train = PatientList.load(path, 'train', age_start, age_stop, age_in_months)
        valid = PatientList.load(path, 'valid', age_start, age_stop, age_in_months)
        test  = PatientList.load(path, 'test',  age_start, age_stop, age_in_months)
        return train, valid, test

    def get_splits(self):
        '''Return splits'''
        return self.train, self.valid, self.test
    
    def get_lengths(self):
        '''Return a dataframe with lengths (# of patients) of the splits (train, valid, test) and total'''
        lengths = [len(self.train), len(self.valid), len(self.test), len(self.train)+len(self.valid)+len(self.test)]
        return pd.DataFrame(lengths, index=['train','valid','test','total'], columns=['lengths'])
    
    def get_label_counts(self, labels):
        '''Get prevalence counts of labels in each split - returns a dataframe with counts for each split and total count'''
        counts = []
        for label in labels:
            train_count = [getattr(self.train[i],label) == 1 for i in range(len(self.train))].count(True)
            valid_count = [getattr(self.valid[i],label) == 1 for i in range(len(self.valid))].count(True)
            test_count  = [getattr(self.test[i],label) == 1 for i in range(len(self.test))].count(True)
            total_count = train_count+valid_count+test_count
            counts.append([train_count, valid_count, test_count, total_count])
        return pd.DataFrame(counts, index=labels, columns=['train','valid','test','total'])
    
    def get_pos_wts(self, labels):
        '''Get positive weights to be used in `nn.BCEWithLogitsLoss`'''
        pos_counts = self.get_label_counts(labels)
        neg_counts = self.get_lengths().transpose().values - pos_counts
        return round(neg_counts / pos_counts)

In [ ]:
show_doc(EHRDataSplits, title_level=3)

<h3 id="EHRDataSplits" class="doc_header"><code>class</code> <code>EHRDataSplits</code><a href="" class="source_link" style="float:right">[source]</a></h3>

> <code>EHRDataSplits</code>(**`path`**, **`age_start`**=*`0`*, **`age_stop`**=*`20`*, **`age_in_months`**=*`False`*)

Class to hold the PatientList splits; defaults to loading 0 to 20 years age span

In [ ]:
show_doc(EHRDataSplits._load_splits)

<h4 id="EHRDataSplits._load_splits" class="doc_header"><code>EHRDataSplits._load_splits</code><a href="__main__.py#L7" class="source_link" style="float:right">[source]</a></h4>

> <code>EHRDataSplits._load_splits</code>(**`path`**, **`age_start`**, **`age_stop`**, **`age_in_months`**)

Load splits of preprocessed [`PatientList`](/lemonpie/preprocessing_transform.html#PatientList)s from persistent store using path

In [ ]:
show_doc(EHRDataSplits.get_splits)

<h4 id="EHRDataSplits.get_splits" class="doc_header"><code>EHRDataSplits.get_splits</code><a href="__main__.py#L14" class="source_link" style="float:right">[source]</a></h4>

> <code>EHRDataSplits.get_splits</code>()

Return splits

In [ ]:
show_doc(EHRDataSplits.get_lengths)

<h4 id="EHRDataSplits.get_lengths" class="doc_header"><code>EHRDataSplits.get_lengths</code><a href="__main__.py#L18" class="source_link" style="float:right">[source]</a></h4>

> <code>EHRDataSplits.get_lengths</code>()

Return a dataframe with lengths (# of patients) of the splits (train, valid, test) and total

In [ ]:
show_doc(EHRDataSplits.get_label_counts)

<h4 id="EHRDataSplits.get_label_counts" class="doc_header"><code>EHRDataSplits.get_label_counts</code><a href="__main__.py#L23" class="source_link" style="float:right">[source]</a></h4>

> <code>EHRDataSplits.get_label_counts</code>(**`labels`**)

Get prevalence counts of labels in each split - returns a dataframe with counts for each split and total count

In [ ]:
show_doc(EHRDataSplits.get_pos_wts)

<h4 id="EHRDataSplits.get_pos_wts" class="doc_header"><code>EHRDataSplits.get_pos_wts</code><a href="__main__.py#L34" class="source_link" style="float:right">[source]</a></h4>

> <code>EHRDataSplits.get_pos_wts</code>(**`labels`**)

Get positive weights to be used in `nn.BCEWithLogitsLoss`

**Tests**

In [ ]:
PATH_1K, LABELS

('/home/vinod/.lemonpie/datasets/synthea/1K',
 ['diabetes', 'stroke', 'alzheimers', 'coronaryheart'])

In [ ]:
splits = EHRDataSplits(PATH_1K)

In [ ]:
splits.get_lengths()

,lengths
train,702
valid,234
test,235
total,1171


In [ ]:
prevalence = splits.get_label_counts(LABELS)
prevalence

,train,valid,test,total
diabetes,43,14,19,76
stroke,30,7,11,48
alzheimers,12,7,6,25
coronaryheart,39,11,11,61


**Cross check with raw**
- Check total counts against raw_csv
- Check split counts against split/raw_csv

In [ ]:
[getattr(splits.train[i],'diabetes') == 1 for i in range(len(splits.train))].count(True)

43

In [ ]:
raw_cnds = pd.read_csv(f'{PATH_1K}/raw_original/conditions.csv', low_memory=False)

In [ ]:
print(raw_cnds[raw_cnds.CODE == 44054006].CODE.count()) #diabetes
print(raw_cnds[raw_cnds.CODE == 230690007].CODE.count()) #stroke
print(raw_cnds[raw_cnds.CODE == 26929004].CODE.count()) #alzheimers
print(raw_cnds[raw_cnds.CODE == 53741008].CODE.count()) #coronary_heart

76
48
25
61


In [ ]:
raw_cnds_train = pd.read_csv(f'{PATH_1K}/raw_split/train/conditions.csv', low_memory=False)
raw_cnds_valid = pd.read_csv(f'{PATH_1K}/raw_split/valid/conditions.csv', low_memory=False)
raw_cnds_test  = pd.read_csv(f'{PATH_1K}/raw_split/test/conditions.csv', low_memory=False)

In [ ]:
print(raw_cnds_train[raw_cnds_train.CODE == 44054006].CODE.count()) #diabetes
print(raw_cnds_valid[raw_cnds_valid.CODE == 44054006].CODE.count()) #diabetes
print(raw_cnds_test [raw_cnds_test.CODE == 44054006].CODE.count()) #diabetes

43
14
19


In [ ]:
prevalence.loc['diabetes'].total

76

In [ ]:
cnd_codes = [44054006, 230690007, 26929004, 53741008]

In [ ]:
for code,name in zip(cnd_codes, LABELS):
    print(code,': ',name)

44054006 :  diabetes
230690007 :  stroke
26929004 :  alzheimers
53741008 :  coronaryheart


In [ ]:
for code, name in zip(cnd_codes, LABELS):
    assert prevalence.loc[name].total == raw_cnds[raw_cnds.CODE == code].CODE.count()
    assert prevalence.loc[name].train == raw_cnds_train[raw_cnds_train.CODE == code].CODE.count()
    assert prevalence.loc[name].valid == raw_cnds_valid[raw_cnds_valid.CODE == code].CODE.count()
    assert prevalence.loc[name].test  == raw_cnds_test [raw_cnds_test.CODE == code]. CODE.count()

## Label

**Labeling** definition in fastai -- some processes need to be run on `train` and **applied** to `valid`

This is completed in preprocessing (vocab & transform) as follows
1. Vocabs created from train data
    - Tokenizing unique values for different record codes & demographic values
    - Calculating mean and std for age
2. Vocabs applied to train, valid and test data
    - With `numericalize` for record codes & demographic values
    - With normalizing of age with the mean / std from train

**Hence labeling in our case will be creating X and y**

- X is the patient object
- y needs to be a tensor made out of - diabetes, stroke, alzheimers, coronaryheart

So **creating the `y` tensor** is simply a matter of ..
1. extracting the values of each of the 4 labels from each `Patient` object 
2. turning it into a `torch.FloatTensor`
3. and stacking them up using `torch.stack`

In [ ]:
tst_y = np.array((True, False, False, True), dtype='float')
torch.from_numpy(tst_y), torch.FloatTensor(tst_y)

(tensor([1., 0., 0., 1.], dtype=torch.float64), tensor([1., 0., 0., 1.]))

2 ways of creating torch tensor from a numpy array, we will stick with the latter

In [ ]:
y = []
for pt in splits.train:
    y.append(torch.FloatTensor(np.array([getattr(pt,label) for label in LABELS], dtype='float')) )

In [ ]:
# y

In [ ]:
y = torch.stack(y)

In [ ]:
y.shape

torch.Size([702, 4])

In [ ]:
y

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

Putting it into a function

In [ ]:
def label_data(patient_ds, labels) -> 'x,y':
    '''Extracts y from patient object, returns x=Patient object, y=tensor of conditions'''
    def _get_y(ds, labels):
        y = []
        for pt in ds:
            y.append( torch.FloatTensor(np.array([getattr(pt,label) for label in labels], dtype='float')) )
        return torch.stack(y)
    
    x, y = patient_ds, _get_y(patient_ds, labels)
    return x,y

In [ ]:
x_train,y_train = label_data(splits.train, LABELS)
x_valid,y_valid = label_data(splits.valid, LABELS)
x_test ,y_test  = label_data(splits.test , LABELS)

In [ ]:
y_train.shape, y_valid.shape, y_test.shape

(torch.Size([702, 4]), torch.Size([234, 4]), torch.Size([235, 4]))

In [ ]:
#export
class LabelEHRData():
    '''Class to hold labeled EHR data splits'''
    def __init__(self, train, valid, test, labels):
        '''Extracts y from patient object, each labelset a tuple of x,y: x=Patient object, y=tensor of conditions'''
        self.x_train, self.y_train = train, self._get_y(train, labels)
        self.x_valid, self.y_valid = valid, self._get_y(valid, labels)
        self.x_test,  self.y_test  = test , self._get_y(test , labels)
        
        self.train = self.x_train, self.y_train
        self.valid = self.x_valid, self.y_valid
        self.test  = self.x_test,  self.y_test
    
    def _get_y(self, ds, labels):
        '''Extract y from each patient object in ds and stack them - ds is dataset containing patient objects'''
        y = []
        for pt in ds:
            y.append( torch.FloatTensor(np.array([getattr(pt,label) for label in labels], dtype='float')) )
        return torch.stack(y)

In [ ]:
show_doc(LabelEHRData, title_level=3)

<h3 id="LabelEHRData" class="doc_header"><code>class</code> <code>LabelEHRData</code><a href="" class="source_link" style="float:right">[source]</a></h3>

> <code>LabelEHRData</code>(**`train`**, **`valid`**, **`test`**, **`labels`**)

Class to hold labeled EHR data splits

In [ ]:
show_doc(LabelEHRData.__init__)

<h4 id="LabelEHRData.__init__" class="doc_header"><code>LabelEHRData.__init__</code><a href="__main__.py#L4" class="source_link" style="float:right">[source]</a></h4>

> <code>LabelEHRData.__init__</code>(**`train`**, **`valid`**, **`test`**, **`labels`**)

Extracts y from patient object, each labelset a tuple of x,y: x=Patient object, y=tensor of conditions

In [ ]:
show_doc(LabelEHRData._get_y)

<h4 id="LabelEHRData._get_y" class="doc_header"><code>LabelEHRData._get_y</code><a href="__main__.py#L14" class="source_link" style="float:right">[source]</a></h4>

> <code>LabelEHRData._get_y</code>(**`ds`**, **`labels`**)

Extract y from each patient object in ds and stack them - ds is dataset containing patient objects

In [ ]:
labeled = LabelEHRData(*splits.get_splits(), LABELS)

## Dataset

[Subclasses](https://pytorch.org/docs/master/data.html?highlight=dataloader#torch.utils.data.Dataset) `torch.utils.data.Dataset`<br>
- that is implements `__len__()` and `__getitem__()`

In [ ]:
#export
class EHRDataset(torch.utils.data.Dataset):
    '''Class to hold a single EHR dataset (holds a tuple of x and y) -- handles lazy vs full loading of dataset on GPU'''
    def __init__(self, x_labeled, y_labeled, lazy_load_gpu=True): 
        '''If `lazy_load_gpu` is `False`, load entire dataset on GPU'''
        if lazy_load_gpu: 
            self.x, self.y = x_labeled, y_labeled
            self.lazy = True
        else:             
            self.x, self.y = [x.to_gpu() for x in x_labeled], y_labeled.to(DEVICE)
            self.lazy = False
            
    def __len__(self): return len(self.x)
   
    def _test_getitem(self, i): return self.x[i],self.y[i]
    
    def __getitem__(self, i): 
        '''If lazy loading, return deep copy of patient object `i`, else entire dataset already on GPU - just return `i`'''
        if self.lazy: 
            return copy.deepcopy(self.x[i]), self.y[i]
        else:
            return self.x[i], self.y[i]

In [ ]:
show_doc(EHRDataset, title_level=3)

<h3 id="EHRDataset" class="doc_header"><code>class</code> <code>EHRDataset</code><a href="" class="source_link" style="float:right">[source]</a></h3>

> <code>EHRDataset</code>(**`x_labeled`**, **`y_labeled`**, **`lazy_load_gpu`**=*`True`*) :: `Dataset`

Class to hold a single EHR dataset (holds a tuple of x and y) -- handles lazy vs full loading of dataset on GPU

In [ ]:
show_doc(EHRDataset.__init__)

<h4 id="EHRDataset.__init__" class="doc_header"><code>EHRDataset.__init__</code><a href="__main__.py#L4" class="source_link" style="float:right">[source]</a></h4>

> <code>EHRDataset.__init__</code>(**`x_labeled`**, **`y_labeled`**, **`lazy_load_gpu`**=*`True`*)

If `lazy_load_gpu` is `False`, load entire dataset on GPU

In [ ]:
show_doc(EHRDataset.__getitem__)

<h4 id="EHRDataset.__getitem__" class="doc_header"><code>EHRDataset.__getitem__</code><a href="__main__.py#L17" class="source_link" style="float:right">[source]</a></h4>

> <code>EHRDataset.__getitem__</code>(**`i`**)

If lazy loading, return deep copy of patient object `i`, else entire dataset already on GPU - just return `i`

Since `Patient` is a custom object and not a typical tensor, we need to handle the behavior for `Dataset`, `DataLoader`, etc to function correctly.
- Memory pinning is a good idea for better performance if lazy loading to GPU
    - [A discussion - pin memory vs full load to GPU](https://discuss.pytorch.org/t/pin-memory-vs-sending-direct-to-gpu-from-dataset/33891)
- So when a DataLoader pins memory on a tensor and copy of the tensor is made on page-locked memory in RAM as opposed to swappable memory which speed up transfers to GPU
    - [A good explanation](https://stackoverflow.com/questions/5736968/why-is-cuda-pinned-memory-so-fast)
- But on custom data type like our `Patient` object, we need to define the behavior
    - [Pytorch docs](https://pytorch.org/docs/stable/data.html#memory-pinning)
- Making a [deep copy](https://docs.python.org/3/library/copy.html) of the `Patient`object to mimick tensor behavior
    - Otherwise, given the Patient holds it's changed tensors, all tensors are CUDA tensors after the first epoch and DL tries to pin memory again and this causes an error (TODO: Need to elaborate)

In [ ]:
def get_ds(x_train,y_train, x_valid,y_valid) -> 'train_ds, valid_ds':
    train_ds,valid_ds = EHRDataset(x_train, y_train), EHRDataset(x_valid, y_valid)
    return train_ds, valid_ds

**Testing Lazy Load**

In [ ]:
train_ds, valid_ds = get_ds(*labeled.train, *labeled.valid)

In [ ]:
len(train_ds), len(valid_ds)

(702, 234)

In [ ]:
len(labeled.train), len(labeled.x_train)

(2, 702)

In [ ]:
assert len(train_ds)==len(labeled.x_train)==len(labeled.y_train)
assert len(valid_ds)==len(labeled.y_valid)==len(labeled.x_valid)

In [ ]:
xb,yb = train_ds[0:5]
xb,yb

([ptid:0ace3e15-8aa4-41c5-8b90-2408285ebcfe, birthdate:1986-04-02, diabetes:False, device:cpu,
  ptid:af1495be-5077-4087-98b1-9ff624c7582c, birthdate:2008-07-17, diabetes:False, device:cpu,
  ptid:f23e12d9-2ec6-4006-b041-ea78d374e9c9, birthdate:2014-09-06, diabetes:False, device:cpu,
  ptid:1968aa31-5fce-461a-9486-6e385a7b75e7, birthdate:1986-04-11, diabetes:False, device:cpu,
  ptid:1211c8ff-ab73-49f3-b2ab-87b7a03f6167, birthdate:1972-03-24, diabetes:False, device:cpu],
 tensor([[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]))

In [ ]:
yb.shape

torch.Size([5, 4])

In [ ]:
xb[0].obs_nums.is_pinned()

False

In [ ]:
train_ds._test_getitem(0)

(ptid:0ace3e15-8aa4-41c5-8b90-2408285ebcfe, birthdate:1986-04-02, diabetes:False, device:cpu,
 tensor([0., 0., 0., 0.]))

## DataLoader - Using Pytorch DataLoader

**Need to define a custom collate function**, because default collate cannot handle list of patient objects in x, gives following error
```
TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class '__main__.Patient'>
```

In [ ]:
valid_ds[0:4]

([ptid:8d1ba4bb-7250-4295-be1c-5d0d423e55f7, birthdate:1957-02-13, diabetes:True, device:cpu,
  ptid:f1921fc3-fdfc-441d-a928-27c18002fedf, birthdate:1909-12-22, diabetes:False, device:cpu,
  ptid:fc4aa89c-e441-4c0b-841f-3d16ffe1b235, birthdate:1981-04-24, diabetes:False, device:cpu,
  ptid:4e0be087-7a33-4655-a9c0-f00f23178ac1, birthdate:1977-02-03, diabetes:False, device:cpu],
 tensor([[1., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]))

In [ ]:
x_tmps,y_tmps = valid_ds[0:4]

In [ ]:
x_tmps

[ptid:8d1ba4bb-7250-4295-be1c-5d0d423e55f7, birthdate:1957-02-13, diabetes:True, device:cpu,
 ptid:f1921fc3-fdfc-441d-a928-27c18002fedf, birthdate:1909-12-22, diabetes:False, device:cpu,
 ptid:fc4aa89c-e441-4c0b-841f-3d16ffe1b235, birthdate:1981-04-24, diabetes:False, device:cpu,
 ptid:4e0be087-7a33-4655-a9c0-f00f23178ac1, birthdate:1977-02-03, diabetes:False, device:cpu]

In [ ]:
y_tmps

tensor([[1., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

**Old collate fns**

**1. removed cuda calls**
```python
def collate(b):
    xs,ys = zip(*b)
    return [x.to_gpu() for x in xs], torch.unsqueeze(torch.tensor(ys), 1).cuda()
```
**2. removed unsqueeze**
```python
def collate(b):
    xs,ys = zip(*b)
    return xs, torch.unsqueeze(torch.tensor(ys), 1)
```

In [ ]:
def collate_ehr(b):
    '''Custom collate function for use in `DataLoader`'''
    xs,ys = zip(*b)
    return xs, torch.stack(ys)

In [ ]:
bs = 2

In [ ]:
def get_dls(train_ds, valid_ds, bs, collate_fn=collate_ehr, lazy=True) -> 'train_dl, valid_dl':
    return(DataLoader(train_ds, batch_size=bs, shuffle=True, collate_fn=collate_fn, pin_memory=lazy),
           DataLoader(valid_ds, batch_size=bs*2, collate_fn=collate_fn, pin_memory=lazy))

In [ ]:
train_dl, valid_dl = get_dls(train_ds, valid_ds, bs)

**Tests - `iter()`, `next()` - Next Batch**

In [ ]:
it = iter(valid_dl)
first_x, first_y = next(it)
second_x, second_y = next(it)

In [ ]:
first_x, first_y

([ptid:8d1ba4bb-7250-4295-be1c-5d0d423e55f7, birthdate:1957-02-13, diabetes:True, device:cpu,
  ptid:f1921fc3-fdfc-441d-a928-27c18002fedf, birthdate:1909-12-22, diabetes:False, device:cpu,
  ptid:fc4aa89c-e441-4c0b-841f-3d16ffe1b235, birthdate:1981-04-24, diabetes:False, device:cpu,
  ptid:4e0be087-7a33-4655-a9c0-f00f23178ac1, birthdate:1977-02-03, diabetes:False, device:cpu],
 tensor([[1., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]))

In [ ]:
first_x[3].med_offsts.is_pinned(), first_y.is_pinned()

(False, True)

In [ ]:
second_x, second_y

([ptid:6d048a56-edb8-4f29-891d-7a84d75a8e78, birthdate:1914-09-05, diabetes:False, device:cpu,
  ptid:4fc76a3b-e39e-4091-a6af-3595e0cb607e, birthdate:1948-06-01, diabetes:False, device:cpu,
  ptid:26ca976d-0b5b-4662-af41-535ff670dd5a, birthdate:2014-09-22, diabetes:False, device:cpu,
  ptid:59486a8b-389b-4355-9df4-edc62bbd1a11, birthdate:1951-10-11, diabetes:False, device:cpu],
 tensor([[0., 0., 1., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]))

In [ ]:
second_x[0].alg_nums

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
second_x[0].alg_nums.is_pinned()

False

**Testing full GPU loading (non-Lazy)**

In [ ]:
train_ds,valid_ds = EHRDataset(*labeled.train, lazy_load_gpu=False), EHRDataset(*labeled.valid, lazy_load_gpu=False)

In [ ]:
xb,yb = train_ds[0:5]
xb,yb

([ptid:0ace3e15-8aa4-41c5-8b90-2408285ebcfe, birthdate:1986-04-02, diabetes:False, device:cuda:0,
  ptid:af1495be-5077-4087-98b1-9ff624c7582c, birthdate:2008-07-17, diabetes:False, device:cuda:0,
  ptid:f23e12d9-2ec6-4006-b041-ea78d374e9c9, birthdate:2014-09-06, diabetes:False, device:cuda:0,
  ptid:1968aa31-5fce-461a-9486-6e385a7b75e7, birthdate:1986-04-11, diabetes:False, device:cuda:0,
  ptid:1211c8ff-ab73-49f3-b2ab-87b7a03f6167, birthdate:1972-03-24, diabetes:False, device:cuda:0],
 tensor([[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]], device='cuda:0'))

In [ ]:
xb[0].demographics.is_pinned()

False

In [ ]:
train_dl, valid_dl = get_dls(train_ds, valid_ds, bs, lazy=False)

In [ ]:
x_tmp, y_tmp = next(iter(valid_dl))

In [ ]:
x_tmp[0].demographics.is_pinned()

False

In [ ]:
x_tmp[0]

ptid:8d1ba4bb-7250-4295-be1c-5d0d423e55f7, birthdate:1957-02-13, diabetes:True, device:cuda:0

In [ ]:
#export
class EHRData:
    '''All encompassing class for EHR data - holds Splits, Labels, Datasets, DataLoaders and provides convenience fns for training and prediction'''
    def __init__(self, path, labels, age_start=0, age_stop=20, age_in_months=False, lazy_load_gpu=True):
        self.path, self.labels = path, labels
        self.age_start, self.age_stop, self.age_in_months = age_start, age_stop, age_in_months
        self.lazy_load_gpu = lazy_load_gpu
    
    def load_splits(self):
        '''Load data splits given dataset path'''
        self.splits = EHRDataSplits(self.path, self.age_start, self.age_stop, self.age_in_months)
    
    def label(self):
        '''Run labeler - i.e. extract y from patient objects'''
        self.labeled = LabelEHRData(*self.splits.get_splits(), self.labels)
        
    def create_datasets(self):
        '''Create `EHRDataset`s'''
        self.train_ds = EHRDataset(*self.labeled.train, self.lazy_load_gpu)
        self.valid_ds = EHRDataset(*self.labeled.valid, self.lazy_load_gpu)
        self.test_ds  = EHRDataset(*self.labeled.test, self.lazy_load_gpu)
        
    def ehr_collate(b):
        '''Custom collate function for use in `DataLoader`'''
        xs,ys = zip(*b)
        return xs, torch.stack(ys)
    
    def create_dls(self, bs, lazy, c_fn=ehr_collate, **kwargs):
        '''Create `DataLoader`s'''
        self.train_dl = DataLoader(self.train_ds, bs, shuffle=True, collate_fn=c_fn, pin_memory=lazy, **kwargs)
        self.valid_dl = DataLoader(self.valid_ds, bs*2, collate_fn=c_fn, pin_memory=lazy, **kwargs)
        self.test_dl  = DataLoader(self.test_ds,  bs*2, collate_fn=c_fn, pin_memory=lazy, **kwargs)
        
    def get_data(self, bs=64, num_workers=0):
        '''Convenience function - returns everything needed for training'''
        self.load_splits()
        self.label()
        self.create_datasets()
        self.create_dls(bs, self.lazy_load_gpu, num_workers=num_workers)

        pos_wts = self.splits.get_pos_wts(self.labels)
        train_pos_wts = torch.Tensor(pos_wts['train'].values)
        valid_pos_wts = torch.Tensor(pos_wts['valid'].values)
        return self.train_dl, self.valid_dl, train_pos_wts, valid_pos_wts

    def get_test_data(self, bs=64, num_workers=0):
        '''Convenience function - returns everything needed for prediction using test data'''
        self.load_splits()
        self.label()
        self.create_datasets()
        self.create_dls(bs, self.lazy_load_gpu, num_workers=num_workers)
                        
        pos_wts = self.splits.get_pos_wts(self.labels)
        test_pos_wts = torch.Tensor(pos_wts['test'].values)
        return self.test_dl, test_pos_wts

In [ ]:
show_doc(EHRData.load_splits)

<h4 id="EHRData.load_splits" class="doc_header"><code>EHRData.load_splits</code><a href="__main__.py#L9" class="source_link" style="float:right">[source]</a></h4>

> <code>EHRData.load_splits</code>()

Load data splits given dataset path

In [ ]:
show_doc(EHRData.label)

<h4 id="EHRData.label" class="doc_header"><code>EHRData.label</code><a href="__main__.py#L13" class="source_link" style="float:right">[source]</a></h4>

> <code>EHRData.label</code>()

Run labeler - i.e. extract y from patient objects

In [ ]:
show_doc(EHRData.create_datasets)

<h4 id="EHRData.create_datasets" class="doc_header"><code>EHRData.create_datasets</code><a href="__main__.py#L17" class="source_link" style="float:right">[source]</a></h4>

> <code>EHRData.create_datasets</code>()

Create [`EHRDataset`](/lemonpie/data.html#EHRDataset)s

In [ ]:
show_doc(EHRData.ehr_collate)

<h4 id="EHRData.ehr_collate" class="doc_header"><code>EHRData.ehr_collate</code><a href="__main__.py#L23" class="source_link" style="float:right">[source]</a></h4>

> <code>EHRData.ehr_collate</code>(**`b`**)

Custom collate function for use in `DataLoader`

In [ ]:
show_doc(EHRData.create_dls)

<h4 id="EHRData.create_dls" class="doc_header"><code>EHRData.create_dls</code><a href="__main__.py#L28" class="source_link" style="float:right">[source]</a></h4>

> <code>EHRData.create_dls</code>(**`bs`**, **`lazy`**, **`c_fn`**=*`ehr_collate`*, **\*\*`kwargs`**)

Create `DataLoader`s

In [ ]:
show_doc(EHRData.get_data)

<h4 id="EHRData.get_data" class="doc_header"><code>EHRData.get_data</code><a href="__main__.py#L34" class="source_link" style="float:right">[source]</a></h4>

> <code>EHRData.get_data</code>(**`bs`**=*`64`*, **`num_workers`**=*`0`*)

Convenience function - returns everything needed for training

In [ ]:
show_doc(EHRData.get_test_data)

<h4 id="EHRData.get_test_data" class="doc_header"><code>EHRData.get_test_data</code><a href="__main__.py#L46" class="source_link" style="float:right">[source]</a></h4>

> <code>EHRData.get_test_data</code>(**`bs`**=*`64`*, **`num_workers`**=*`0`*)

Convenience function - returns everything needed for prediction using test data

## Export -

In [ ]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_basics.ipynb.
Converted 01_preprocessing_clean.ipynb.
Converted 02_preprocessing_vocab.ipynb.
Converted 03_preprocessing_transform.ipynb.
Converted 04_data.ipynb.
Converted 05_metrics.ipynb.
Converted 06_learn.ipynb.
Converted 07_models.ipynb.
Converted 08_experiment.ipynb.
Converted 99_quick_walkthru.ipynb.
Converted 99_running_exps.ipynb.
Converted index.ipynb.
